### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [2]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [3]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [4]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [5]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,42624359,142002,79085,51530,0,0,1134,5,2025-04-03 22:00,2025-04-03 22:00,0,1,5299,3,1
34,42624234,2901,79085,3,0,0,1134,6,2025-04-04 19:35,2025-04-04 19:35,2,1,7572,4,1
35,42624503,90417,79085,51530,0,0,1135,0,2025-04-05 22:00,2025-04-05 22:00,0,2,3697,6,1
36,42410283,79085,79866,1349,5,70,1135,1,2025-04-06 21:59,2025-04-06 21:59,0,0,12093,4,1
37,42629918,78026,79085,51530,0,0,1135,2,2025-04-07 22:00,2025-04-07 22:00,3,1,5819,5,1
38,42630368,79536,79085,51530,0,0,1135,3,2025-04-08 22:00,2025-04-08 22:00,0,1,2349,4,1
39,42410295,79085,89202,1349,6,70,1135,4,2025-04-09 21:59,2025-04-09 21:59,7,0,6180,4,1


In [6]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [7]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [8]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [9]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,142002,46546,30726,0,0,1,0,0,Knudde 433,8,8,7,42624359
1,79085,23647,39467,0,16,1,0,0,4-3-3aRU,10,9,11,42624359
0,2901,34672,35684,0,7,3,0,0,skrzy532hura+2,8,5,7,42624234
1,79085,35301,34289,1,9,4,3,1,5-4-1a,7,7,7,42624234
0,90417,36476,35862,1,3,2,0,0,GG-22,9,8,11,42624503
1,79085,33477,34091,0,8,1,1,0,5-3-2c,10,10,9,42624503
0,79085,34281,34584,1,9,2,0,0,4-3-3aRU,9,9,10,42410283
1,79866,35673,35370,0,8,0,0,0,Haikyuu!,10,9,10,42410283
0,78026,29215,36698,0,17,5,2,0,Ammyalr,12,11,11,42629918
1,79085,40742,33259,0,2,3,2,0,5-4-1a,8,9,10,42629918


In [10]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [11]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1672

### Estatísticas da Liga

In [12]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [13]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [14]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,23775,6,18,6,0,0,20,4,18516020006001,1349
1,10520,6,16,5,1,0,23,3,16520023005011,1349
2,79085,6,13,4,1,1,19,4,13515019004004,1349
3,23755,6,12,4,0,2,23,8,12515023004010,1349
4,78671,6,12,4,0,2,18,10,12508018004008,1349
5,124139,6,10,3,1,2,14,7,10507014003006,1349
6,79866,6,8,2,2,2,15,6,8509015002009,1349
7,122169,6,8,2,2,2,9,7,8502009002005,1349
8,89569,6,3,1,0,5,5,12,3493005001007,1349
9,79665,6,3,1,0,5,2,30,3472002001002,1349


In [15]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [16]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [17]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [18]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [19]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [20]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()